<H1><b>Assignment: Segmenting and Clustering Neighborhoods in Toronto</b></H1>
<H2>Problem 2</H2>

In [81]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [82]:
#scrape the following Wikipedia page link
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source=requests.get(url).text

In [83]:
soup=bs(source, 'xml')

In [84]:
#Finding the table from the xml page
Table=soup.find('table')

In [85]:
#Creating TorontoDF dataframe
colum_names=['PostalCode', 'Borough', 'Neighbourhood']
TorontoDF=pd.DataFrame(columns=colum_names)

In [86]:
#Assembling the table data to the newly created dataframe using for loop
for tr in Table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        TorontoDF.loc[len(TorontoDF)]= row_data

In [87]:
TorontoDF.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<H2>Data Cleaning</H2>

In [88]:
indexes=TorontoDF[TorontoDF['Borough']=='Not assigned'].index
TorontoDF=TorontoDF.drop(indexes)
TorontoDF.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [89]:
temp_df=TorontoDF.groupby('PostalCode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)

In [91]:
df_merge = pd.merge(TorontoDF, temp_df, on='PostalCode')

In [93]:
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [94]:
df_merge.drop_duplicates(inplace=True)

In [95]:
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [96]:
df_merge.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [97]:
df_merge.shape

(103, 3)

In [105]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [106]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [107]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [108]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='PostalCode')

In [109]:
geo_data=geo_merged[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]

In [110]:
geo_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
